In [2]:
import numpy as np
from scipy.sparse import csr_matrix as sparse_mat
import pickle
import sys
from sklearn.cross_validation import train_test_split
import math

In [6]:
# returns the non-zero entries from the sparse matrix
def non_zero(sparse_matrix):
    return np.transpose(sparse_matrix.nonzero())

## Loading Ratings Matrix Pickle

In [5]:
ratings = pickle.load(open('pickles/ratings_matrix.p', 'rb'))

In [4]:
ratings.shape

(552339, 77445)

---
## Splitting the Dataset into Training, Validation and Testing Sets
##### Do Not Run This Everytime

- Training Set: 70%  
- Test Set: 30%  

In [31]:
training = []
testing = []
sample = []

for j, business in enumerate(ratings.T):
    indices = non_zero(business)
    indices = [[idx[1], j] for idx in indices]
    if len(indices) > 10:
        train, test = train_test_split(indices, test_size=0.3, random_state=42)
        training.extend(train)
        testing.extend(test)
        if np.random.uniform(low=0, high=1) <= 0.1:
            sample.extend(train)
    else:
        training.extend(indices)
        if np.random.uniform(low=0, high=1) <= 0.1:
            sample.extend(indices)

#### Saving split data into pickles, so that we can evaluate all models on the same training, validation and testing sets

In [32]:
training_sample, validation_sample = train_test_split(sample, test_size=0.3, random_state=42)

In [35]:
pickle.dump(training, open('pickles/training_indices.p', 'wb'))
pickle.dump(testing, open('pickles/testing_indices.p', 'wb'))

In [40]:
pickle.dump(training_sample, open('pickles/training_samples.p', 'wb'))
pickle.dump(validation_sample, open('pickles/validation_samples.p', 'wb'))

---

### Load training, validation and testing data directly

In [3]:
training = pickle.load(open('pickles/training_indices.p', 'rb'))
testing = pickle.load(open('pickles/testing_indices.p', 'rb'))

##### Load the training and validation sample directly if required

In [6]:
training_sample = pickle.load(open('pickles/training_samples.p', 'rb'))
validation_sample = pickle.load(open('pickles/validation_samples.p', 'rb'))

## The Latent Factor Model, without considering reviews:

In [12]:
def train_latent_factors(R, training_indices, k=2, eta=1e-2, lamb=1e-2, threshold=1e-6, iterations=2500):
    m, n = R.shape
    U = np.random.uniform(-1, 1, (m, k))
    B = np.random.uniform(-1, 1, (n, k))
    i = 0
    prev = 0
    curr = sys.maxint
    T = len(training_indices)
    while i < iterations and abs(curr - prev) > threshold:
        prev = curr
        cost = 0
        for idx in training_indices:
            epsilon = R[idx[0], idx[1]] - np.dot(U[idx[0]], B[idx[1]].T)
            U[idx[0], :] = U[idx[0], :] + eta * (epsilon * B[idx[1]] - lamb * U[idx[0], :])
            B[idx[1], :] = B[idx[1], :] + eta * (epsilon * U[idx[0]] - lamb * B[idx[1], :])
            cost += (R[idx[0], idx[1]] - np.dot(U[idx[0]], B[idx[1]].T )) ** 2 + \
                                    lamb * (np.linalg.norm(U[idx[0]])**2 + np.linalg.norm(B[idx[1]])**2)

        cost = math.sqrt(cost) / T
        curr = cost
        print curr
        i += 1
        
    print "Total Iterations: ", i
    return U, B

#### Run for user, business and review loading directly

In [6]:
user_map = pickle.load(open('pickles/users_map.p', 'rb'))
business_map = pickle.load(open('pickles/business_map.p', 'rb'))
reviews = pickle.load(open('pickles/reviews.p', 'rb'))

#### Global and Individual Baselines have to be adjusted before training

In [17]:
baselines = pickle.load(open('pickles/baselines.p', 'rb'))
alpha = baselines['alpha']
beta_users = baselines['beta_users']
beta_business = baselines['beta_business']

In [18]:
# removing baselines from ratings data
for idx in non_zero(ratings):
    ratings[idx[0], idx[1]] -= (alpha + beta_users[idx[0]] + beta_business[idx[1]])

##### Train the model on the training set

In [19]:
U, B = train_latent_factors(ratings, training, k=10, iterations=50)

0.000850987737661
0.000715884893602
0.000621437228679
0.000551492144223
0.000497422119349
0.000454202038018
0.000418706776023
0.000388890672492
0.0003633590322
0.000341129454906
0.000321491573338
0.000303920778515
0.00028802302697
0.000273498307584
0.000260115728536
0.000247696077583
0.000236099327363
0.0002252154965
0.000214957840416
0.000205257693694
0.000196060506569
0.000187322761477
0.000179009550455
0.000171092657916
0.000163549036647
0.000156359594491
0.000149508229676
0.000142981066994
0.000136765857163
0.00013085150906
0.000125227730074
0.000119884754202
0.00011481314104
0.000110003631786
0.000105447050826
0.000101134243525
9.7056042538e-05
9.32032562931e-05
8.95666743826e-05
8.61370854656e-05
8.29053039624e-05
7.98622024088e-05
7.69987468342e-05
7.43060329834e-05
7.17753216215e-05
6.93980715571e-05
6.71659693821e-05
6.50709552622e-05
6.31052444006e-05
6.12613440465e-05
Total Iterations:  50


In [20]:
reviews.head(5)

,business_id,review_id,stars,text,user_id
0,5UmKMjUEUNdYWqANhGckJw,Ya85v4eqdd6k9Od8HbQjyA,4,"Mr Hoagie is an institution. Walking in, it do...",PUFPaY9KxDAcGqfsorJp3Q
1,5UmKMjUEUNdYWqANhGckJw,KPvLNJ21_4wbYNctrOwWdQ,5,Excellent food. Superb customer service. I mis...,Iu6AxdBYGR4A0wspR9BYHA
2,5UmKMjUEUNdYWqANhGckJw,fFSoGV46Yxuwbr3fHNuZig,5,Yes this place is a little out dated and not o...,auESFwWvW42h6alXgFxAXQ
3,UsFtqoBl7naz8AVUBZMjQQ,Di3exaUCFNw1V4kSNW5pgA,5,All the food is great here. But the best thing...,uK8tzraOp4M5u3uYrqIBXg
4,UsFtqoBl7naz8AVUBZMjQQ,0Lua2-PbqEQMjD9r89-asw,3,We checked this place out this past Monday for...,I_47G-R2_egp7ME5u_ltew


### Validate the model on a random sample of 10000 points:

- Sample the first 10% businesses from the total data.
- Split the sample into training and validation only (70 - 30)
- Train for range of $\lambda$, and compute errors on validation.
- The $\lambda$ with best error is saved and used for training the full dataset

The full data will be split into 70-30 train test

In [11]:
total, _ = reviews.shape

In [12]:
training_sample, _ = train_test_split(training, test_size=10000/total, random_state=42)

In [14]:
def predict(u_idx, b_idx, U, B):
    return np.dot(U[u_idx], B[b_idx].T) + alpha + beta_users[u_idx] + beta_business[b_idx]

In [15]:
def errors(test_set, U, B, ratings=ratings):
    error = 0
    for idx in test_set:
        u_idx = idx[0]
        b_idx = idx[1]
        error += (ratings[u_idx, b_idx] - predict(u_idx, b_idx, U, B)) ** 2
    return math.sqrt(error)/len(test_set)

In [25]:
def validate(training_sample, validation_sample, ratings=ratings, start=0.005, end=0.02, steps=0.001):
    best_error = sys.maxint
    best_lambda = 0
    for lam in np.arange(start, end, steps):
        U, B = train_latent_factors(ratings, training_sample, k=10, iterations=50, lamb=lam)
        error = errors(validation_sample, U, B)
        if (error < best_error):
            best_error = error
            best_lambda = lam
    return best_lambda

In [33]:
best_lambda = validate(training_sample, validation_sample)

0.000929771713575
0.000809912206745
0.000708718656482
0.000622561279765
0.000548716003906
0.000485083173571
0.000430009823869
0.000382172020538
0.000340494044881
0.000304091203343
0.000272228339811
0.000244289101153
0.000219752754325
0.000198176421919
0.000179181278232
0.000162441686856
0.000147676553501
0.000134642367447
0.000123127544255
0.000112947781296
0.000103942209204
9.59701748288e-05
8.8908530197e-05
8.26493310739e-05
7.70978705598e-05
7.21709896033e-05
6.77956187136e-05
6.39075145106e-05
6.04501618533e-05
5.73738176816e-05
5.46346768657e-05
5.21941435786e-05
5.001819426e-05
4.80768202802e-05
4.63435400849e-05
4.47949719826e-05
4.34104599071e-05
4.21717454769e-05
4.10626805314e-05
4.00689750611e-05
3.91779760927e-05
3.83784736534e-05
3.76605304242e-05
3.70153321242e-05
3.64350560408e-05
3.59127554467e-05
3.54422579315e-05
3.50180759259e-05
3.46353279122e-05
3.42896690067e-05
Total Iterations:  50
0.000919286305693
0.000805145717077
0.000708137899161
0.000625072141376
0.0005535

In [8]:
best_lambda = 0.016

In [19]:
# train the model with final set of parameters
U, B = train_latent_factors(ratings, training, k=10, lamb=best_lambda)
test_errors_latent_factors = errors(testing, U, B)
print test_errors_latent_factors

0.00107650690234
0.000966999869108
0.000914038466699
0.000878796516848
0.000852442636361
0.000831378828296
0.000813780547687
0.000798598738376
0.000785178538547
0.000773086838525
0.00076202473755
0.000751779154929
0.000742194273175
0.000733153785658
0.00072456938581
0.00071637304058
0.000708511653738
0.000700943292049
0.000693634464855
0.000686558132479
0.00067969223029
0.00067301856441
0.000666521979299
0.000660189726379
0.000654010982494
0.000647976480553
0.000642078224432
0.000636309267281
0.000630663537653
0.000625135701799
0.000619721053396
0.000614415424199
0.000609215110696
0.000604116813095
0.000599117583886
0.000594214783898
0.000589406044318
0.000584689233516
0.000580062427815
0.000575523885557
0.000571072023937
0.000566705398231
0.000562422683049
0.000558222655386
0.000554104179222
0.000550066191516
0.000546107689458
0.000542227718877
0.00053842536372
0.00053469973656
0.000531049970062
0.000527475209379
0.000523974605413
0.00052054730892
0.00051719246538
0.000513909210624
0.

In [21]:
test_errors_latent_factors = errors(testing, U, B)
print "Mean Squared Error of Latent Factor Model: ", test_errors_latent_factors

Mean Squared Error of Latent Factor Model:  0.00550474466419


In [20]:
pickle.dump(U, open('latent_factor_U.p', 'wb'))
pickle.dump(B, open('latent_factor_B.p', 'wb'))